In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import os
import scipy.sparse as sparse
from test import *

from itertools import chain
from pathlib import *
from math import ceil
from sqlalchemy import create_engine
from pandarallel import pandarallel

from test import *
from helper import *
from tree_weight import *
from time import time
from main import make_graph_from_tree
from multiprocessing import *

from typing import List
%matplotlib inline

In [2]:
pandarallel.initialize(nb_workers=cpu_count() - 1, progress_bar=True)

INFO: Pandarallel will run on 3 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [11]:
image_folder = 'graphs/'
tree = 'benchmark_models/golomb/trees/09.sqlite'
info_df = to_df(tree, 'info').set_index('NodeID')
nodes_df = to_df(tree, 'nodes').set_index('NodeID')

mappings = pd.read_csv('benchmark_models/golomb/trees/09.csv', header=None, quotechar="'")
map_dict = mappings.set_index(0).to_dict()[1]
map_dict.update(mappings.set_index(1).to_dict()[0])

In [3]:
image_folder = 'graphs/'
tree = 'benchmark_models/mario/trees/mario_easy_3.sqlite'
info_df = to_df(tree, 'info').set_index('NodeID')
nodes_df = to_df(tree, 'nodes').set_index('NodeID')

mappings = pd.read_csv(tree.replace('.sqlite', '.csv'), header=None, quotechar="'")
map_dict = mappings.set_index(0).to_dict()[1]
map_dict.update(mappings.set_index(1).to_dict()[0])

print(nodes_df.shape)

# if 'SubtreeSize' not in nodes_df:
#     calculate_subtree_size(nodes_df)
#     to_sqlite(nodes_df, tree)

# valid_df = nodes_df[nodes_df['Status'] != 3]

def get_split_var(row, mappings=None):
    node_idx = row.name
    par_idx = valid_df.loc[node_idx, 'ParentID']
    if par_idx == - 1:
        return None
    cands, _, par_domain, children_domain = find_split_variable(par_idx, valid_df, info_df, mappings)
    return cands[0] if len(cands) == 1 else None

def get_parent_column(column: str, df: pd.DataFrame) -> pd.Series:
    has_root = 1 if 0 == df.index[0] else 0
    j = df.iloc[has_root:, :].reset_index().set_index('ParentID')[['NodeID', column]]
    j.loc[:, column] = df.loc[df.iloc[has_root:, :]['ParentID'], column]    
    return j.reset_index().set_index('NodeID')[column]
    
# valid_df.loc[:, 'SplitVariable'] = valid_df.apply(get_split_var, axis=1, mappings=map_dict)
# valid_df.loc[:, 'SubtreeSizeWeight'] = valid_df['SubtreeSize'] / get_parent_column('SubtreeSize', valid_df)
# valid_df.loc[:, 'SearchSpaceWeight'] = valid_df['SearchSpaceNodeWeight'] / get_parent_column('SearchSpaceNodeWeight', valid_df)

(31283, 11)


In [30]:
valid_df = nodes_df[nodes_df['Status'] != 3]

# use vectorized string to get names of labels
node_label = valid_df['Label'].str.split('=', expand=True)[0].str.strip('! ').rename('LabelVarname')
node_label = node_label[node_label != '']
# use indexing to map nodes' labels names to info's domain names
info_label = node_label.reset_index().set_index('LabelVarname')
info_label['LabelName'] = mappings.set_index(1)[0]
info_label = info_label.set_index('NodeID')
# assign parent id to extract parent's domain for split variable
info_label['ParentID'] = valid_df['ParentID']
info_label = info_label.drop_duplicates('ParentID')
info_label = info_label.set_index('ParentID').reset_index().rename(columns={'ParentID': 'NodeID'}).set_index('NodeID')

In [31]:
info_label

,LabelName
NodeID,
0,X_INTRODUCED_0_
1,X_INTRODUCED_2_
3,X_INTRODUCED_3_
5,X_INTRODUCED_4_
7,X_INTRODUCED_5_
...,...
31271,X_INTRODUCED_3_
31273,X_INTRODUCED_6_
31274,X_INTRODUCED_3_


In [32]:
info_df['Label'] = info_label['LabelName']
# get parent's domain size of split variables of children
parent_domain_size = info_df[info_df['Label'].notna()].parallel_apply(lambda row: len(parse_info_string(row['Info'], early_stop=row['Label'])), axis=1).rename('DomainSize')

In [34]:
# (valid_df['Label'].str.find('!') > 0)
parent_domain_size

NodeID
0        694
1        694
2        693
3        684
4        684
        ... 
31268      1
31271      1
31273      1
31274      1
31278      1
Name: DomainSize, Length: 15641, dtype: int64

In [122]:
k = 0.9
res_df = pd.DataFrame.copy(valid_df.iloc[1:, :])
res_df['Weight'] = np.random.random(res_df.shape[0])
parent_mean = res_df.groupby('ParentID').mean()['Weight']
parent_count = res_df.groupby('ParentID').count()['Weight']

# index swap
res_df = res_df.reset_index().set_index('ParentID')
res_df.loc[:, 'Mean'] = parent_mean
res_df.loc[:, 'Count'] = parent_count
res_df = res_df.reset_index().set_index('NodeID')
res_df.loc[:, 'Weight'] = res_df['Weight'] - res_df['Mean'] + k / res_df['Count']
j = res_df.groupby(['ParentID']).sum()['Weight']
j[abs(j - k) > 1e-10]

Series([], Name: Weight, dtype: float64)

In [119]:
weights = valid_df[['ParentID', 'HasUnequalSplit']].reset_index().set_index('ParentID').iloc[1:,:]
weights.loc[:, 'ParentDomainSize'] = parent_domain_size
weights = weights.reset_index().set_index('NodeID')
weights['Weight'] = 1 / weights['ParentDomainSize'] + weights['HasUnequalSplit'] * (1 - 2 / weights['ParentDomainSize'])
weights.drop(columns)

,ParentID,HasUnequalSplit,ParentDomainSize,Weight
NodeID,,,,
1,0,False,694,0.001441
2,0,True,694,0.998559
3,1,False,694,0.001441
4,1,True,694,0.998559
5,3,False,684,0.001462
...,...,...,...,...
31278,31274,True,1,0.000000
31279,31278,False,1,1.000000
31280,31278,True,1,0.000000


In [48]:
weights

array([0.00144092, 0.99855908, 0.00144092, ..., 0.        , 1.        ,
       0.        ])

In [10]:
domains

,Info,label
NodeID,,
0,"{'objective': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1...",NaN
1,"{'objective': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1...",X_INTRODUCED_0_
2,"{'objective': {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, ...",X_INTRODUCED_0_
3,"{'objective': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1...",X_INTRODUCED_2_
4,"{'objective': {10, 11, 12, 13, 14, 15, 16, 17,...",X_INTRODUCED_2_
...,...,...
31278,"{'objective': {693}, 'fuel': {1542, 1543, 1544...",X_INTRODUCED_3_
31279,"{'objective': {693}, 'fuel': {1984, 1985, 1986...",X_INTRODUCED_3_
31280,"{'objective': {693}, 'fuel': {1938, 1939, 1940...",X_INTRODUCED_3_


In [14]:
nodeSplitVarDomain = domains.iloc[1:,:].parallel_apply(lambda r: len(r['Info'][r['label']]), axis=1)

ModuleNotFoundError: No module named 'ipywidgets'

In [ ]:
domains['parentLabel'] = get_parent_column('label', domains)

In [ ]:
domains

In [ ]:
print(info_df.shape)